In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1660 Ti (UUID: GPU-2a3d1dcb-c220-642c-91e9-cb51a54e9757)


In [2]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

%matplotlib inline

In [2]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"

dataset = tf.keras.utils.get_file("../datasets/stack_overflow_dataset/stack_overflow_16k", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')
dataset
dataset_dir = os.path.join(os.path.dirname(dataset), 'stack_overflow_dataset')

In [3]:
train_dir = "./train"
test_dir = "./test"

os.listdir(train_dir)

['csharp', 'java', 'javascript', 'python']

In [4]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir, 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    test_dir, 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [5]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Review b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default constru

In [6]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])
print("Label 1 corresponds to", raw_train_ds.class_names[2])
print("Label 1 corresponds to", raw_train_ds.class_names[3])

Label 0 corresponds to csharp
Label 1 corresponds to java
Label 1 corresponds to javascript
Label 1 corresponds to python


In [7]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [8]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [9]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [10]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [11]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'"set blank to quit on exception? i\'m using blank 3..i\'ve been looking around for an answer to this, but i haven\'t found it yet. basically, i\'m running several blank scripts into a game engine, and each script has its own entry point...i\'d rather not add try: except blocks through all of my code, so i was wondering if it\'s at all possible to tell blank to quit (or perhaps assign a custom function to that ""callback"") on finding its first error, regardless of where or what it found? ..currently, the game engine will continue after finding and hitting an error, making it more difficult than necessary to diagnose issues since running into one error may make a subsequent script not work (as it relies on variables that the error-ing script set, for example). any ideas? ..i know that i could redirect the console to a file to allow for easier scrolling, but just capturing the first error and stopping the game prematurely would be really useful...okay, a couple of extr

In [11]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  scissors
 313 --->  source
Vocabulary size: 10000


In [12]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [13]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, 16),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(4)])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 4)                 68        
                                                                 
Total params: 160,084
Trainable params: 160,084
Non-trainable params: 0
__________________________________________________

In [15]:
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
epochs = 30
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/30
200/200 [==============================] - 3s 13ms/step - loss: 1.3805 - accuracy: 0.3414 - val_loss: 1.3721 - val_accuracy: 0.4431
Epoch 2/30
200/200 [==============================] - 1s 3ms/step - loss: 1.3543 - accuracy: 0.4605 - val_loss: 1.3356 - val_accuracy: 0.4944
Epoch 3/30
200/200 [==============================] - 0s 2ms/step - loss: 1.3032 - accuracy: 0.5323 - val_loss: 1.2766 - val_accuracy: 0.5750
Epoch 4/30
200/200 [==============================] - 1s 3ms/step - loss: 1.2326 - accuracy: 0.5836 - val_loss: 1.2055 - val_accuracy: 0.6106
Epoch 5/30
200/200 [==============================] - 0s 2ms/step - loss: 1.1550 - accuracy: 0.6308 - val_loss: 1.1306 - val_accuracy: 0.6475
Epoch 6/30
200/200 [==============================] - 0s 2ms/step - loss: 1.0763 - accuracy: 0.6662 - val_loss: 1.0610 - val_accuracy: 0.6900
Epoch 7/30
200/200 [==============================] - 0s 2ms/step - loss: 1.0071 - accuracy: 0.7075 - val_loss: 0.9980 - val_accuracy: 0.7113
Epoch

In [17]:
loss, accuracy = model.evaluate(val_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

50/50 [==============================] - 0s 1ms/step - loss: 0.5405 - accuracy: 0.8044
Loss:  0.5404895544052124
Accuracy:  0.8043749928474426


In [18]:
model_1 = tf.keras.Sequential([
  layers.Embedding(max_features + 1, 16),
  layers.GlobalAveragePooling1D(),
  layers.Dense(4)])
model_1.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 4)                 68        
                                                                 
Total params: 160,084
Trainable params: 160,084
Non-trainable params: 0
__________________________________________________

In [19]:
epochs = 30
history_1 = model_1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/30
200/200 [==============================] - 1s 2ms/step - loss: 1.3756 - accuracy: 0.3556 - val_loss: 1.3623 - val_accuracy: 0.4506
Epoch 2/30
200/200 [==============================] - 0s 2ms/step - loss: 1.3369 - accuracy: 0.4931 - val_loss: 1.3123 - val_accuracy: 0.5362
Epoch 3/30
200/200 [==============================] - 0s 2ms/step - loss: 1.2710 - accuracy: 0.5533 - val_loss: 1.2400 - val_accuracy: 0.5975
Epoch 4/30
200/200 [==============================] - 0s 2ms/step - loss: 1.1876 - accuracy: 0.6119 - val_loss: 1.1590 - val_accuracy: 0.6456
Epoch 5/30
200/200 [==============================] - 0s 2ms/step - loss: 1.0999 - accuracy: 0.6698 - val_loss: 1.0801 - val_accuracy: 0.6850
Epoch 6/30
200/200 [==============================] - 0s 2ms/step - loss: 1.0161 - accuracy: 0.7100 - val_loss: 1.0085 - val_accuracy: 0.7144
Epoch 7/30
200/200 [==============================] - 0s 2ms/step - loss: 0.9400 - accuracy: 0.7402 - val_loss: 0.9455 - val_accuracy: 0.7294
Epoch 

In [20]:
loss, accuracy = model_1.evaluate(val_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

50/50 [==============================] - 0s 679us/step - loss: 0.5276 - accuracy: 0.8037
Loss:  0.5275871157646179
Accuracy:  0.8037499785423279


In [23]:
model_2 = tf.keras.Sequential([
  layers.Embedding(max_features + 1, 128),
  layers.LSTM(units=64, return_sequences=True),
  layers.LSTM(64),
  layers.Dense(64, activation='relu'),
  layers.Dense(4)])
model_2.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])
model_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 128)         1280128   
                                                                 
 lstm_2 (LSTM)               (None, None, 64)          49408     
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 4)                 260       
                                                                 
Total params: 1,366,980
Trainable params: 1,366,980
Non-trainable params: 0
_________________________________________________________________


In [24]:
epochs = 30
history_2 = model_2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/30
200/200 [==============================] - 45s 214ms/step - loss: 1.3852 - accuracy: 0.2583 - val_loss: 1.3833 - val_accuracy: 0.2463
Epoch 2/30
200/200 [==============================] - 43s 213ms/step - loss: 1.3641 - accuracy: 0.2870 - val_loss: 1.3644 - val_accuracy: 0.2606
Epoch 3/30
200/200 [==============================] - 43s 214ms/step - loss: 1.3388 - accuracy: 0.3052 - val_loss: 1.2865 - val_accuracy: 0.3219
Epoch 4/30
200/200 [==============================] - 45s 226ms/step - loss: 1.2332 - accuracy: 0.3931 - val_loss: 1.2768 - val_accuracy: 0.4100
Epoch 5/30
200/200 [==============================] - 45s 226ms/step - loss: 1.2707 - accuracy: 0.3884 - val_loss: 1.2965 - val_accuracy: 0.3644
Epoch 6/30
200/200 [==============================] - 45s 224ms/step - loss: 1.3070 - accuracy: 0.3648 - val_loss: 1.3074 - val_accuracy: 0.3350
Epoch 7/30
200/200 [==============================] - 45s 223ms/step - loss: 1.3057 - accuracy: 0.3628 - val_loss: 1.3087 - val_ac

In [25]:
loss, accuracy = model_2.evaluate(val_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

50/50 [==============================] - 3s 61ms/step - loss: 1.1822 - accuracy: 0.7513
Loss:  1.1821742057800293
Accuracy:  0.7512500286102295


In [26]:
model_3 = tf.keras.Sequential([
  layers.Embedding(max_features + 1, 128),
  layers.GRU(64, return_sequences=True),
  layers.LSTM(64, return_sequences=True),
  layers.GRU(64),
  layers.Dense(64, activation='relu'),
  layers.Dense(4)])
model_3.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])
model_3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 128)         1280128   
                                                                 
 gru (GRU)                   (None, None, 64)          37248     
                                                                 
 lstm_4 (LSTM)               (None, None, 64)          33024     
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 4)                 260       
                                                                 
Total params: 1,379,780
Trainable params: 1,379,780
No

In [27]:
epochs = 30
history_3 = model_3.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/30
200/200 [==============================] - 74s 352ms/step - loss: 1.3849 - accuracy: 0.2680 - val_loss: 1.3840 - val_accuracy: 0.2425
Epoch 2/30
200/200 [==============================] - 69s 347ms/step - loss: 1.3660 - accuracy: 0.2781 - val_loss: 1.3730 - val_accuracy: 0.2544
Epoch 3/30
200/200 [==============================] - 70s 351ms/step - loss: 1.3391 - accuracy: 0.2956 - val_loss: 1.3935 - val_accuracy: 0.2500
Epoch 4/30
200/200 [==============================] - 73s 363ms/step - loss: 1.2886 - accuracy: 0.3342 - val_loss: 1.1854 - val_accuracy: 0.4325
Epoch 5/30
200/200 [==============================] - 72s 358ms/step - loss: 1.1300 - accuracy: 0.4347 - val_loss: 1.1242 - val_accuracy: 0.4737
Epoch 6/30
200/200 [==============================] - 71s 356ms/step - loss: 1.0004 - accuracy: 0.5263 - val_loss: 0.9776 - val_accuracy: 0.5119
Epoch 7/30
200/200 [==============================] - 71s 353ms/step - loss: 0.7841 - accuracy: 0.6803 - val_loss: 0.6995 - val_ac

In [28]:
loss, accuracy = model_3.evaluate(val_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

50/50 [==============================] - 4s 90ms/step - loss: 1.1444 - accuracy: 0.7781
Loss:  1.1443883180618286
Accuracy:  0.778124988079071


In [29]:
model_4 = tf.keras.Sequential([
  layers.Embedding(max_features + 1, 128),
  layers.LSTM(128, return_sequences=True),
  layers.GRU(64),
  layers.Dense(64, activation='relu'),
  layers.Dense(4, activation='softmax')])
model_4.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])
model_4.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 128)         1280128   
                                                                 
 lstm_5 (LSTM)               (None, None, 128)         131584    
                                                                 
 gru_2 (GRU)                 (None, 64)                37248     
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 4)                 260       
                                                                 
Total params: 1,453,380
Trainable params: 1,453,380
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 30
history_4 = model_4.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/30


C:\Users\Admin\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


200/200 [==============================] - 52s 246ms/step - loss: 1.3856 - accuracy: 0.2658 - val_loss: 1.3842 - val_accuracy: 0.2438
Epoch 2/30
200/200 [==============================] - 50s 251ms/step - loss: 1.3302 - accuracy: 0.3231 - val_loss: 1.3195 - val_accuracy: 0.3469
Epoch 3/30
200/200 [==============================] - 51s 253ms/step - loss: 1.2762 - accuracy: 0.3844 - val_loss: 1.2631 - val_accuracy: 0.3831
Epoch 4/30
200/200 [==============================] - 51s 257ms/step - loss: 1.2146 - accuracy: 0.4025 - val_loss: 1.4679 - val_accuracy: 0.4181
Epoch 5/30
200/200 [==============================] - 50s 249ms/step - loss: 1.2609 - accuracy: 0.3909 - val_loss: 1.1818 - val_accuracy: 0.4381
Epoch 6/30
200/200 [==============================] - 48s 240ms/step - loss: 1.2092 - accuracy: 0.3758 - val_loss: 1.2289 - val_accuracy: 0.3562
Epoch 7/30
200/200 [==============================] - 48s 242ms/step - loss: 1.1900 - accuracy: 0.3898 - val_loss: 1.4047 - val_accuracy: 0.2